In [ ]:
filepath = "gdrive/My Drive/.wav/"
input_filepath = "gdrive/My Drive/.mp4/"
output_filepath="gdrive/My Drive/transcribed_files2/"


In [ ]:
!pip install pydub
!pip install SpeechRecognition
!pip install --upgrade google-cloud-speech
!pip install ffmpy
!pip install libav-tools

     |████████████████████████████████| 32.8MB 137kB/s 
     |████████████████████████████████| 92kB 3.2MB/s 


  Created wheel for ffmpy: filename=ffmpy-0.2.2-cp36-none-any.whl size=4602 sha256=9841c8413ad4df77d3e01e5b29e9dd7c475d90e2849722c9a076802284ee78ee
  Stored in directory: /root/.cache/pip/wheels/16/28/57/96aff0d874198125b03f542d854e7ebdc61a56b09a4d49de6a
Successfully built ffmpy
ERROR: Could not find a version that satisfies the requirement libav-tools (from versions: none)
ERROR: No matching distribution found for libav-tools


In [ ]:
!gcloud config set project weighty-flux-127101


Updated property [core/project].


In [ ]:
from pydub import AudioSegment
import io
import os
import ffmpy
from google.cloud import speech_v1p1beta1 as speech
from google.cloud.speech_v1p1beta1 import enums
from google.cloud.speech_v1p1beta1 import types
import wave
from google.cloud import storage
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#filepath = "gdrive/My Drive/check_point/"
#output_filepath = "gdrive/My Drive/transcribed_files/"
#saver = tf.train.Saver(max_to_keep = 100)

In [ ]:
import os
#FIXME: EDIT CREDENTIALS GO TO DOWNLOAD KEYS FROM LEAH'S PROJECT
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "gdrive/My Drive/credentials_gcp/MyWallBreakers-0b82f601b198.json"
#check=os.listdir("gdrive/My Drive/check_point")
#saver.restore(session, "gdrive/My Drive/checkpoint/epoch{}/chatbot.ckpt".format(0))


In [ ]:
def mp5_to_wav(audio_file_name):
  for filename in os.listdir(input_filepath):
    actual_filename = filename[:-4]
    if (filename.endswith(".mp4")):
      os.system('ffmpeg -i {} -acodec pcm_s16le -ar 16000 {}.wav'.format(audio_file_name, actual_filename))
    else:
      continue

In [ ]:
def mp3_to_wav(audio_file_name):
  print(":)")
  if audio_file_name.split('.')[1] == 'mp3':    
    sound = AudioSegment.from_mp3(audio_file_name)
    audio_file_name = audio_file_name.split('.')[0] + '.wav'
    sound.export(audio_file_name, format="wav")

In [ ]:

def mp4_to_wav():
  print(os.listdir(input_filepath))
  for filename in os.listdir(input_filepath):
    print(filename)
    actual_filename = filename[:-4]
    
    if (filename.endswith(".mp4")):
      
      sound = AudioSegment.from_file(input_filepath + filename)
      print('made it')
      audio_file_name = actual_filename +'.wav'
      sound.export(filepath + audio_file_name, format = "wav")
      print(filename)
    else:
      continue

    

In [ ]:
mp4_to_wav()

['zoom_2.mp4', 'zoom_3.mp4', 'zoom_4.mp4', 'zoom_5.mp4', 'zoom_1.mp4']
zoom_2.mp4
made it
zoom_2.mp4
zoom_3.mp4
made it
zoom_3.mp4
zoom_4.mp4
made it
zoom_4.mp4
zoom_5.mp4
made it
zoom_5.mp4
zoom_1.mp4
made it
zoom_1.mp4


In [ ]:
def frame_rate_channel(audio_file_name):
    with wave.open(audio_file_name, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        channels = wave_file.getnchannels()
        return frame_rate,channels

In [ ]:
def stereo_to_mono(audio_file_name):
    sound = AudioSegment.from_wav(audio_file_name)
    sound = sound.set_channels(1)
    sound.export(audio_file_name, format="wav")

In [ ]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

In [ ]:
def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.delete()

In [ ]:
def google_transcribe(audio_file_name):
    
    file_name = filepath + audio_file_name
    print(file_name)
    #mp3_to_wav(file_name)
    #mp4_to_wav(file_name)

    # The name of the audio file to transcribe
    
    frame_rate, channels = frame_rate_channel(file_name)
    
    if channels > 1:
        stereo_to_mono(file_name)
    
    bucket_name = 'calls-audio-files'
    source_file_name = filepath + audio_file_name
    destination_blob_name = audio_file_name
    
    upload_blob(bucket_name, source_file_name, destination_blob_name)
    
    gcs_uri = 'gs://calls-audio-files/' + audio_file_name
    transcript = ''
    
    client = speech.SpeechClient()
    audio = types.RecognitionAudio(uri=gcs_uri)

    config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=frame_rate,
    language_code='en-US',
    enable_speaker_diarization=True, 
    diarization_speaker_count=2
    )

    # Detects speech in the audio file
    operation = client.long_running_recognize(config, audio)
    response = operation.result(timeout=10000)
    result = response.results[-1]
    words_info = result.alternatives[0].words
    
    tag=1
    speaker=""

    for word_info in words_info:
        if word_info.speaker_tag==tag:
            speaker=speaker+" "+word_info.word
        else:
            transcript += "speaker {}: {}".format(tag,speaker) + '\n'
            tag=word_info.speaker_tag
            speaker=""+word_info.word

    transcript += "speaker {}: {}".format(tag,speaker)
    
    delete_blob(bucket_name, destination_blob_name)
    return transcript

In [ ]:
def write_transcripts(transcript_filename,transcript):
    f= open(output_filepath + transcript_filename,"w+")
    f.write(transcript)
    f.close() 

In [ ]:
#gdrive/My Drive/checkpoint/epoch{}/chatbot.ckpt".format(0)
if __name__ == "__main__":
    for audio_file_name in os.listdir(filepath):
        print(audio_file_name)
        exists = os.path.isfile(output_filepath + audio_file_name.split('.')[0] + '.txt')
        if exists:
            print("!!!!!????? :/")
            pass
        else:
            transcript = google_transcribe(audio_file_name)
            print(transcript)
            print(":)")
            transcript_filename = audio_file_name.split('.')[0] + '.txt'
            write_transcripts(output_filepath + transcript_filename,transcript)

zoom_2.wav
gdrive/My Drive/.wav/zoom_2.wav
speaker 1:  really only three people so far I know that the the announcement didn't go out Andrea wasn't sure if we were having this today because we had because we had the office hours would be so last night but technically with this is if this is still on the counter so I don't know why you wouldn't we are great great office hours with Visa I changed the the permission so now everybody should be able to access this but if we don't get more people onto this then I'm just going to go ahead and reschedule us from for next week there's no reason why we can't do that so we're going to wait maybe 5 more minutes and see if we get more than two people can you tell me what the syntax error is anybody see what my syntax error is nobody I'm assuming the people is defined know that that is the Syntax for PHP and you absolutely can use a logical operator and an Australian PHP OU Dynamic typing for the win yeah I would assume and also that would assume th

FileNotFoundError: ignored